# <span style="color:red;">1. Ownership Information on Windows</span>

**In Windows**, ownership is often represented differently than on Unix systems. While `os.stat()` still provides `st_uid` and `st_gid`, these may not directly translate to meaningful user or group names without using `Windows-specific` APIs like `win32security`.

It is possible to retrieve ownership information using Python’s `os` and `win32api` modules.

In [5]:
# 1. Retrieve Ownership Information using win32security

import win32security

def get_file_owner(file_path):
    sd = win32security.GetFileSecurity(file_path, win32security.OWNER_SECURITY_INFORMATION)
    owner_sid   = sd.GetSecurityDescriptorOwner()
    owner, _, _ = win32security.LookupAccountSid(None, owner_sid)
    
    print(f"Owner: {owner}")


get_file_owner('test_file.txt')

Owner: AshmanMalhotra


**⬆️ It shows the owner of the file `test_file.txt`.**

**Insights:**    
- The `win32security` module is part of the `pywin32` library, which provides access to many Windows APIs for file and security management.
- On Windows, the owner is determined based on the security identifier (SID) associated with the file.

# <span style="color:red;">2. Creation and Modification Times (Windows-Specific)</span>     
Windows handles file creation time (`st_ctime`) differently from Unix. On Windows, `st_ctime` represents the file's creation time, while on Unix it represents the inode change time.

In [7]:
import os
import time

file_path = 'test_file.txt'

# Using os.stat()
stat_info         = os.stat(file_path)
creation_time     = stat_info.st_ctime
modification_time = stat_info.st_mtime

print("Creation time:", time.ctime(creation_time))
print("Last modified:", time.ctime(modification_time))

# Using os.path for convenience
print("\nCreation time:", time.ctime(os.path.getctime(file_path)))
print("Last modified:", time.ctime(os.path.getmtime(file_path)))

Creation time: Sun Aug 25 18:23:23 2024
Last modified: Mon Aug 26 00:36:31 2024

Creation time: Sun Aug 25 18:23:23 2024
Last modified: Mon Aug 26 00:36:31 2024


**Tips:**     
- The `time.ctime()` function converts a timestamp to a readable format (e.g., `Wed Aug 17 15:35:29 2023`).
- On Windows, the creation time retrieved from `os.path.getctime()` accurately represents when the file was created.

# <span style="color:red;">3. Practical Examples and Use Cases for Windows</span>     

## <span style="color:blue;">A. File Size and Creation Time (Windows)</span>

### Example 1: Getting the File Size and Creation Time (Windows)

In [8]:
import os
import time

def get_file_info(file_path):
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        creation_time = os.path.getctime(file_path)
        
        print(f"File Size: {file_size} bytes")
        print(f"Creation Time: {time.ctime(creation_time)}")
    else:
        print("File does not exist.")


get_file_info('test_file.txt')

File Size: 20 bytes
Creation Time: Sun Aug 25 18:23:23 2024


## <span style="color:blue;">B. File Permissions (Windows)</span>

### Example 1: Checking File Permissions (Windows)      
On Windows, the `os.access()` method can still be used to check `read`, `write` and `execute` permissions. However, it’s worth noting that permissions on Windows are managed differently from Unix, relying on ACLs (Access Control Lists) rather than the traditional Unix `rwx` model.

In [9]:
import os

file_path = 'test_file.txt'


# Check if the file is readable
if os.access(file_path, os.R_OK):
    print("File is readable.")
else:
    print("File is not readable.")


# Check if the file is writable
if os.access(file_path, os.W_OK):
    print("File is writable.")
else:
    print("File is not writable.")

File is readable.
File is not writable.


### Example 2: Finding the Oldest File in a Directory (Windows)     
The following example scans a directory to find the oldest file based on creation time:

In [12]:
import os
import time

def find_oldest_file(directory):
    oldest_file = None
    oldest_time = float('inf')             # Start with a very large timestamp

    for root, dirs, files in os.walk(directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            creation_time = os.path.getctime(file_path)
            
            if creation_time < oldest_time:
                oldest_time = creation_time
                oldest_file = file_path

    if oldest_file:
        print(f"Oldest file: {oldest_file}")
        print(f"\nCreation time: {time.ctime(oldest_time)}")
    else:
        print("No files found.")


find_oldest_file(os.getcwd())

Oldest file: C:\Users\AshmanMalhotra\Desktop\Course\AshmanTutsGH\Python Tutorials - Level 1\Chapter 14 - Working with Files and Directories\01 Introduction.ipynb

Creation time: Wed Aug 14 01:29:36 2024


**We can also pass the different directory path to track files inside the given directory.**

### Example 3: Creating a File Listing with Metadata (Windows)
This example generates a list of files in a directory along with their size, creation time and permissions:

In [14]:
import os
import time

def list_files_with_metadata(directory):
    for root, dirs, files in os.walk(directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            stat_info = os.stat(file_path)

            # File size
            file_size = stat_info.st_size

            # Permissions (using os.access)
            readable = os.access(file_path, os.R_OK)
            writable = os.access(file_path, os.W_OK)
            executable = os.access(file_path, os.X_OK)

            # Timestamps
            creation_time = time.ctime(os.path.getctime(file_path))
            modification_time = time.ctime(os.path.getmtime(file_path))

            print(f"File: {file_name}")
            print(f"  Size: {file_size} bytes")
            print(f"  Readable: {readable}")
            print(f"  Writable: {writable}")
            print(f"  Executable: {executable}")
            print(f"  Creation Time: {creation_time}")
            print(f"  Last Modified: {modification_time}")
            print("-" * 50)
            print()


            
list_files_with_metadata(os.getcwd())

File: 01 Introduction.ipynb
  Size: 5528 bytes
  Readable: True
  Writable: True
  Executable: True
  Creation Time: Wed Aug 14 01:29:36 2024
  Last Modified: Sun Aug 25 15:49:51 2024
--------------------------------------------------

File: 02 Common Functions and Attributes.ipynb
  Size: 123833 bytes
  Readable: True
  Writable: True
  Executable: True
  Creation Time: Wed Aug 14 01:34:21 2024
  Last Modified: Sun Aug 25 15:22:32 2024
--------------------------------------------------

File: 03 File and Directory Traversal.ipynb
  Size: 17457 bytes
  Readable: True
  Writable: True
  Executable: True
  Creation Time: Thu Aug 15 13:38:28 2024
  Last Modified: Thu Aug 15 16:49:33 2024
--------------------------------------------------

File: 04-01 File and Directory Metadata with Python's os Module.ipynb
  Size: 48025 bytes
  Readable: True
  Writable: True
  Executable: True
  Creation Time: Thu Aug 15 16:49:41 2024
  Last Modified: Mon Aug 26 17:26:18 2024
---------------------------

**Additional Pointers:**   
- On Windows, you can use the `win32api` and `win32security` modules for more granular control over file security and ownership.
- The `os.walk()` function is highly useful for recursively scanning directories and listing files, making it easier to gather metadata for all files in a directory.
- Use `pathlib` for a more object-oriented approach to file handling. It works seamlessly across platforms and provides higher-level abstractions.

**Ownership information includes:**
- **User ID (UID):** The owner of the file.
- **Group ID (GID):** The group associated with the file.    

This information can be accessed using `os.stat()`.

In [2]:
import os

file_path = 'test_file.txt'
stat_info = os.stat(file_path)

# Getting UID and GID
uid = stat_info.st_uid
gid = stat_info.st_gid

print(f"Owner's User ID: {uid}")
print(f"Owner's Group ID: {gid}")

Owner's User ID: 0
Owner's Group ID: 0


In [3]:
import os
import win32security

def get_file_owner(file_path):
    sd = win32security.GetFileSecurity(file_path, win32security.OWNER_SECURITY_INFORMATION)
    owner_sid = sd.GetSecurityDescriptorOwner()
    owner, _, _ = win32security.LookupAccountSid(None, owner_sid)
    
    print(f"Owner: {owner}")

get_file_owner('test_file.txt')

Owner: AshmanMalhotra
